# ERAD

ERAD is a free, open-source Python toolkit for computing energy resilience measures in the face of hazards like earthquakes and flooding. It uses graph database to store data and perform computation at the household level for a variety of critical services that are connected by power distribution network. It uses asset fragility curves, which are functions that relate hazard severity to survival probability for power system assets including cables, transformers, substations, roof-mounted solar panels, etc. recommended in top literature. Programs like undergrounding, microgrid, and electricity backup units for critical infrastructures may all be evaluated using metrics and compared across different neighborhoods to assess their effects on energy resilience.

In [2]:
pip install git+https://github.com/NREL/erad.git@al/erad_v1

  Cloning https://github.com/NREL/erad.git (to revision al/erad_v1) to /private/var/folders/_7/k4nd0c9s5z914hjjvkpygcg80000gr/T/pip-req-build-grvy7674
  Running command git clone --filter=blob:none --quiet https://github.com/NREL/erad.git /private/var/folders/_7/k4nd0c9s5z914hjjvkpygcg80000gr/T/pip-req-build-grvy7674
  Running command git checkout -b al/erad_v1 --track origin/al/erad_v1
  Switched to a new branch 'al/erad_v1'
  branch 'al/erad_v1' set up to track 'origin/al/erad_v1'.
  Resolved https://github.com/NREL/erad.git to commit 1c9fe623d2c58f301e321a6ec09d04462ff29d61
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for NREL-erad: filename=nrel_erad-0.0.1-py3-none-any.whl size=22172 sha256=daafb89295637836e2d519d14af8d4b21406c6d593c56976af8d8dff377aa02c
  Stored in directory: /private/var/folders/_7/k4nd0c9s5z914hjjvkpygcg80000gr/T/pip-ephem-wheel-cache-_g210olf/wheels/63/03/6

We start by loading a sample GDM system using the gdmloader.

In [3]:
from gdm.distribution import DistributionSystem
from gdmloader.constants import GCS_CASE_SOURCE
from gdmloader.source import SystemLoader

gdm_loader = SystemLoader()
gdm_loader.add_source(GCS_CASE_SOURCE)

distribution_system: DistributionSystem = gdm_loader.load_dataset(
    source_name=GCS_CASE_SOURCE.name, 
    system_type=DistributionSystem, 
    dataset_name="p5r",
)
distribution_system.name = "p5r"

/opt/homebrew/Caskroom/miniconda/base/envs/gdm2/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: Ellipsis is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


ERAD Asset and Hazard systems are also infrasys systems. This means these class also have all the functionality to serialize and deserialize, add conconents and model validation. An `AssetSystem` can be build directly from a given GDM system using the `from_gdm` method. Once built, an instance of `HarzardSimulator` can be created by passing the asset system. Finally, the simualation can be run by calling the `run` method and passing an instance of `HazardSystem`. This system may  have definations of multiple hazards.

In [4]:
from erad.models.asset import Asset
from erad.runner import HarzardSimulator
from erad.systems.asset_system import AssetSystem
from erad.systems.hazard_system import HazardSystem

asset_system =  AssetSystem.from_gdm(distribution_system)
hazard_scenario = HarzardSimulator(asset_system=asset_system)
hazard_scenario.run(hazard_system=HazardSystem.earthquake_example())
for component in asset_system.iter_all_components():
    component.pprint()
    break

2025-05-02 10:25:32.444 | INFO     | erad.models.hazard.common:<module>:16 - Erad database not found. Downloading from Google Cloud.
2025-05-02 10:25:47.130 | INFO     | erad.models.hazard.common:<module>:23 - Download complete...
2025-05-02 10:25:47.770 | WARNING  | erad.runner:run:37 - No HazardFragilityCurves definations found in the passed HazardSystem using default curve definations


Asset(
    name='source',
    distribution_asset=UUID('9db89dca-068e-44aa-8338-39f63cdadb2d'),
    asset_type=<AssetTypes.substation: 0>,
    height=<Quantity(3, 'meter')>,
    latitude=36.60102672210813,
    longitude=-120.91292875147646,
    asset_state=[
        AssetState(
            name='',
            timestamp=datetime.datetime(2025, 5, 2, 10, 25, 47, 769987),
            wind_speed=None,
            flood_velocity=None,
            flood_depth=None,
            fire_boundary_dist=None,
            peak_ground_velocity=SpeedProbability(
                name='',
                survival_probability=0.5300401162709448,
                speed=<Quantity(43.7665689, 'centimeter / second')>
            ),
            peak_ground_acceleration=AccelerationProbability(
                name='',
                survival_probability=0.5027933876698256,
                acceleration=<Quantity(1.52785696, 'meter / second ** 2')>
            ),
            survival_probability=0.2665006656607766
        )
    ]
)